In [ ]:
print("all ok")

all ok


In [ ]:
!pip install confluent_kafka

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 14.0 MB/s eta 0:00:00


In [ ]:
import argparse
from uuid import uuid4
from six.moves import input
from confluent_kafka import Producer
from confluent_kafka.serialization import StringSerializer, SerializationContext, MessageField
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.json_schema import JSONSerializer
#from confluent_kafka.schema_registry import *
import pandas as pd
from typing import List

In [ ]:
FILE_PATH = "/content/dataset_gps.csv"

In [ ]:
import pandas as pd

In [ ]:
pd.read_csv("/content/dataset_gps.csv")

,timestamp,latitude,longitude,elevation,accuracy,bearing,speed_meters_per_second,satellites,provider,hdop,vdop,pdop,geoidheight,ageofdgpsdata,dgpsid,activity,battery,annotation,distance_meters,elapsed_time_seconds
0,1.577221e+09,-27.717803,-51.098857,979.758061,16.0,173.99516,0.201648,12,gps,1.0,1.6,1.9,3.6,NaN,NaN,NaN,81,NaN,0.000000,0.000
1,1.577221e+09,-27.717810,-51.098875,988.071687,12.0,170.69283,0.087510,12,gps,0.9,1.4,1.7,3.6,NaN,NaN,NaN,81,NaN,1.905784,1.002
2,1.577221e+09,-27.717818,-51.098886,988.190402,12.0,255.83438,0.116968,12,gps,0.9,1.5,1.7,3.6,NaN,NaN,NaN,81,NaN,1.372245,1.003
3,1.577221e+09,-27.717820,-51.098893,989.539177,16.0,308.75323,0.049259,12,gps,0.9,1.4,1.6,3.6,NaN,NaN,NaN,81,NaN,0.720180,0.988
4,1.577221e+09,-27.717869,-51.098902,986.509513,16.0,NaN,0.000000,12,gps,0.8,1.3,1.5,3.6,NaN,NaN,NaN,81,NaN,5.482620,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1572,1.577222e+09,-27.717743,-51.099119,980.994608,4.0,131.76976,1.037004,22,gps,0.7,1.0,1.2,3.6,NaN,NaN,NaN,75,NaN,1.032037,1.000
1573,1.577222e+09,-27.717749,-51.099115,980.963261,6.0,129.76077,0.451316,22,gps,0.7,1.0,1.2,3.6,NaN,NaN,NaN,75,NaN,0.754157,1.000
1574,1.577222e+09,-27.717746,-51.099114,981.304869,6.0,129.74986,0.014915,22,gps,0.7,1.0,1.2,3.6,NaN,NaN,NaN,75,NaN,0.289941,1.000
1575,1.577222e+09,-27.717743,-51.099115,981.537893,4.0,129.74384,0.014796,22,gps,0.7,1.0,1.2,3.6,NaN,NaN,NaN,75,NaN,0.335361,1.000


In [ ]:
df = pd.read_csv("/content/dataset_gps.csv")

In [ ]:
df

,timestamp,latitude,longitude,elevation,accuracy,bearing,speed_meters_per_second,satellites,provider,hdop,vdop,pdop,geoidheight,ageofdgpsdata,dgpsid,activity,battery,annotation,distance_meters,elapsed_time_seconds
0,1.577221e+09,-27.717803,-51.098857,979.758061,16.0,173.99516,0.201648,12,gps,1.0,1.6,1.9,3.6,NaN,NaN,NaN,81,NaN,0.000000,0.000
1,1.577221e+09,-27.717810,-51.098875,988.071687,12.0,170.69283,0.087510,12,gps,0.9,1.4,1.7,3.6,NaN,NaN,NaN,81,NaN,1.905784,1.002
2,1.577221e+09,-27.717818,-51.098886,988.190402,12.0,255.83438,0.116968,12,gps,0.9,1.5,1.7,3.6,NaN,NaN,NaN,81,NaN,1.372245,1.003
3,1.577221e+09,-27.717820,-51.098893,989.539177,16.0,308.75323,0.049259,12,gps,0.9,1.4,1.6,3.6,NaN,NaN,NaN,81,NaN,0.720180,0.988
4,1.577221e+09,-27.717869,-51.098902,986.509513,16.0,NaN,0.000000,12,gps,0.8,1.3,1.5,3.6,NaN,NaN,NaN,81,NaN,5.482620,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1572,1.577222e+09,-27.717743,-51.099119,980.994608,4.0,131.76976,1.037004,22,gps,0.7,1.0,1.2,3.6,NaN,NaN,NaN,75,NaN,1.032037,1.000
1573,1.577222e+09,-27.717749,-51.099115,980.963261,6.0,129.76077,0.451316,22,gps,0.7,1.0,1.2,3.6,NaN,NaN,NaN,75,NaN,0.754157,1.000
1574,1.577222e+09,-27.717746,-51.099114,981.304869,6.0,129.74986,0.014915,22,gps,0.7,1.0,1.2,3.6,NaN,NaN,NaN,75,NaN,0.289941,1.000
1575,1.577222e+09,-27.717743,-51.099115,981.537893,4.0,129.74384,0.014796,22,gps,0.7,1.0,1.2,3.6,NaN,NaN,NaN,75,NaN,0.335361,1.000


In [ ]:
df.columns

Index(['timestamp', 'latitude', 'longitude', 'elevation', 'accuracy',
       'bearing', 'speed_meters_per_second', 'satellites', 'provider', 'hdop',
       'vdop', 'pdop', 'geoidheight', 'ageofdgpsdata', 'dgpsid', 'activity',
       'battery', 'annotation', 'distance_meters', 'elapsed_time_seconds'],
      dtype='object')

In [ ]:
df.dtypes

timestamp                  float64
latitude                   float64
longitude                  float64
elevation                  float64
accuracy                   float64
bearing                    float64
speed_meters_per_second    float64
satellites                   int64
provider                    object
hdop                       float64
vdop                       float64
pdop                       float64
geoidheight                float64
ageofdgpsdata              float64
dgpsid                     float64
activity                   float64
battery                      int64
annotation                 float64
distance_meters            float64
elapsed_time_seconds       float64
dtype: object

In [ ]:
columns = ['timestamp', 'latitude', 'longitude', 'elevation', 'accuracy',
       'bearing', 'speed_meters_per_second', 'satellites', 'provider', 'hdop',
       'vdop', 'pdop', 'geoidheight', 'ageofdgpsdata', 'dgpsid', 'activity',
       'battery', 'annotation', 'distance_meters', 'elapsed_time_seconds']

In [ ]:
API_KEY = 'FONZTCNU5SKLIFQV'
ENDPOINT_SCHEMA_URL  = 'https://psrc-epkz2.ap-southeast-2.aws.confluent.cloud'
API_SECRET_KEY = 'M+B+eR6JVmdpjbCjV1p0axL7JHC7WN7GRsZGxoqcgIl0U+qlf658j7PHnqbPiqqP'
BOOTSTRAP_SERVER = 'pkc-l7pr2.ap-south-1.aws.confluent.cloud:9092'
SECURITY_PROTOCOL = 'SASL_SSL'
SSL_MACHENISM = 'PLAIN'
SCHEMA_REGISTRY_API_KEY = '7ZEBNQLNHWFRUKKV'
SCHEMA_REGISTRY_API_SECRET = '3MrOlFBaEsSazWr3gKSY96IBumx6Rx/E0PO4fXS3mcuXZkQDit4PWGn7Q6iMH3kS'

In [ ]:
def sasl_conf():

    sasl_conf = {'sasl.mechanism': SSL_MACHENISM,
                 # Set to SASL_SSL to enable TLS support.
                #  'security.protocol': 'SASL_PLAINTEXT'}
                'bootstrap.servers':BOOTSTRAP_SERVER,
                'security.protocol': SECURITY_PROTOCOL,
                'sasl.username': API_KEY,
                'sasl.password': API_SECRET_KEY
                }
    return sasl_conf

In [ ]:
def schema_config():
    return {'url':ENDPOINT_SCHEMA_URL,

    'basic.auth.user.info':f"{SCHEMA_REGISTRY_API_KEY}:{SCHEMA_REGISTRY_API_SECRET}"

    }

In [ ]:
class Car:
    def __init__(self,record:dict):
        for k,v in record.items():
            setattr(self,k,v)

        self.record=record

    @staticmethod
    def dict_to_car(data:dict,ctx):
        return Car(record=data)

    def __str__(self):
        return f"{self.record}"

In [ ]:
#def get_car_instance(file_path):
#    df=pd.read_csv(file_path)
#    df=df.iloc[:,1:]
 #   cars:List[Car]=[]
 #   for data in df.values:
 #       car=Car(dict(zip(columns,data)))
 #       cars.append(car)
 #       yield car

In [ ]:
def get_car_instance(file_path):
    cars:List[Car]=[]
    for data in df.values:
        car_dict = {key: value for key, value in zip(columns, data)}
        car=Car(car_dict)
        cars.append(car)
    return cars

In [ ]:
get_car_instance('/content/dataset_gps.csv')

 ...]

In [ ]:
def car_to_dict(car:Car, ctx):
    """
    Returns a dict representation of a User instance for serialization.
    Args:
        user (User): User instance.
        ctx (SerializationContext): Metadata pertaining to the serialization
            operation.
    Returns:
        dict: Dict populated with user attributes to be serialized.
    """

    # User._address must not be serialized; omit from dict
    return car.record

In [ ]:
def delivery_report(err, msg):
    """
    Reports the success or failure of a message delivery.
    Args:
        err (KafkaError): The error that occurred on None on success.
        msg (Message): The message that was produced or failed.
    """

    if err is not None:
        print("Delivery failed for User record {}: {}".format(msg.key(), err))
        return
    print('User record {} successfully produced to {} [{}] at offset {}'.format(
        msg.key(), msg.topic(), msg.partition(), msg.offset()))

In [ ]:
FILE_PATH

'/content/dataset_gps.csv'

In [ ]:
df.dtypes

timestamp                  float64
latitude                   float64
longitude                  float64
elevation                  float64
accuracy                   float64
bearing                    float64
speed_meters_per_second    float64
satellites                   int64
provider                    object
hdop                       float64
vdop                       float64
pdop                       float64
geoidheight                float64
ageofdgpsdata              float64
dgpsid                     float64
activity                   float64
battery                      int64
annotation                 float64
distance_meters            float64
elapsed_time_seconds       float64
dtype: object

In [ ]:
def main(topic):

    schema_str = """
   {
  "$id": "http://example.com/myURI.schema.json",
  "$schema": "http://json-schema.org/draft-07/schema#",
  "additionalProperties": false,
  "description": "Sample schema to help you get started.",
  "properties": {
    "accuracy": {
      "description": "genericaccuracy",
      "type": "number"
    },
    "activity": {
      "description": "genericactivity",
      "type": ["string","null"]
    },
    "ageofdgpsdata": {
      "description": "genericageofdgpsdata",
      "type": "string"
    },
    "annotation": {
      "description": "genericannotation",
      "type": "string"
    },
    "battery": {
      "description": "genericbattery",
      "type": "string"
    },
    "bearing": {
      "description": "genericbearing",
      "type": "string"
    },
    "dgpsid": {
      "description": "genericdgpsid",
      "type": "string"
    },
    "distance_meters": {
      "description": "genericdistance_meters",
      "type": "string"
    },
    "elapsed_time_seconds": {
      "description": "genericelapsed_time_seconds",
      "type": "string"
    },
    "elevation": {
      "description": "genericelevation",
      "type": "string"
    },
    "geoidheight": {
      "description": "genericgeoidheight",
      "type": "string"
    },
    "hdop": {
      "description": "generichdop",
      "type": "string"
    },
    "latitude": {
      "description": "genericlatitude",
      "type": "string"
    },
    "longitude": {
      "description": "genericlongitude",
      "type": "string"
    },
    "pdop": {
      "description": "genericpdop",
      "type": "string"
    },
    "provider": {
      "description": "genericprovider",
      "type": "string"
    },
    "satellites": {
      "description": "genericsatellites",
      "type": "string"
    },
    "speed_meters_per_second": {
      "description": "genericspeed_meters_per_second",
      "type": "string"
    },
    "timestamp": {
      "description": "generictimestamp",
      "type": "string"
    },
    "vdop": {
      "description": "genericvdop",
      "type": "string"
    }
  },
  "title": "SampleRecord",
  "type": "object"
}
    """
    schema_registry_conf = schema_config()
    schema_registry_client = SchemaRegistryClient(schema_registry_conf)

    string_serializer = StringSerializer('utf_8')
    json_serializer = JSONSerializer(schema_str, schema_registry_client, car_to_dict)

    producer = Producer(sasl_conf())

    print("Producing user records to topic {}. ^C to exit.".format(topic))
    #while True:
        # Serve on_delivery callbacks from previous calls to produce()
    producer.poll(0.0)
    try:
        for car in get_car_instance(file_path=FILE_PATH):

            print(car)
            producer.produce(topic=topic,
                            key=string_serializer(str(uuid4()), car_to_dict),
                            value=json_serializer(car, SerializationContext(topic, MessageField.VALUE)),
                            on_delivery=delivery_report)
            break
    except KeyboardInterrupt:
        pass
    except ValueError:
        print("Invalid input, discarding record...")
        pass

    print("\nFlushing records...")
    producer.flush()

main("topic_0")


Producing user records to topic topic_0. ^C to exit.
{'timestamp': 1577220691.481, 'latitude': -27.717803461511323, 'longitude': -51.09885741313454, 'elevation': 979.7580610380106, 'accuracy': 16.0, 'bearing': 173.99516, 'speed_meters_per_second': 0.20164786, 'satellites': 12, 'provider': 'gps', 'hdop': 1.0, 'vdop': 1.6, 'pdop': 1.9, 'geoidheight': 3.6, 'ageofdgpsdata': nan, 'dgpsid': nan, 'activity': nan, 'battery': 81, 'annotation': nan, 'distance_meters': 0.0, 'elapsed_time_seconds': 0.0}


SerializationError: nan is not of type 'string', 'null'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.0 MB/s eta 0:00:00


In [ ]:
from fastavro import schemaless_writer

In [ ]:
!pip install jsonschema

In [ ]:
import jsonschema

In [ ]:
schema = jsonschema.Draft7Validator.check_schema(schema_str)

NameError: name 'schema_str' is not defined

In [ ]:
class Sensor:
  def __init__(self,):
    self.sensor1 = 3
    self.sensor2 = 4
  def to_dict(self):
    return self.__dict__

In [ ]:
sensor = Sensor()

In [ ]:
sensor.__dict__

{'sensor1': 3, 'sensor2': 4}

In [ ]:
sensor.to_dict()

{'sensor1': 3, 'sensor2': 4}

In [ ]:
[{"name":"aryan","desigantion":"intern"}]

[{'name': 'aryan', 'desigantion': 'intern'}]

In [ ]:
emp_info = {"name":"aryan","desigantion":"intern"}

In [ ]:
class Emp:

  def __init__(self,record:dict):

    for attr_name,values in record.items():
      setattr(self,attr_name,values)

  @classmethod
  def get_object(record):
    return Emp(record)

In [ ]:
emp = Emp(emp_info)

In [ ]:
emp.name

'aryan'

In [ ]:
emp.desigantion

'intern'

In [ ]:
## practise

In [ ]:
class ACsensor:

  def __init__(self,temp,hum,pressure,pos,flow,vib):
    self.temp=temp
    self.hum = hum
    self.pressure= pressure
    self.pos = pos
    self.flow = flow
    self.vib = vib

  def to_dict(self):
    return self.__dict__

In [ ]:
ACsensor(temp=23,hum=12,pressure=34,pos=98,flow="normal",vib="good")

In [ ]:
ac_reading = ACsensor(temp=23,hum=12,pressure=34,pos=98,flow="normal",vib="good")

In [ ]:
ac_reading.to_dict()

{'temp': 23,
 'hum': 12,
 'pressure': 34,
 'pos': 98,
 'flow': 'normal',
 'vib': 'good'}

In [ ]:
df

,timestamp,latitude,longitude,elevation,accuracy,bearing,speed_meters_per_second,satellites,provider,hdop,vdop,pdop,geoidheight,ageofdgpsdata,dgpsid,activity,battery,annotation,distance_meters,elapsed_time_seconds
0,1.577221e+09,-27.717803,-51.098857,979.758061,16.0,173.99516,0.201648,12,gps,1.0,1.6,1.9,3.6,NaN,NaN,NaN,81,NaN,0.000000,0.000
1,1.577221e+09,-27.717810,-51.098875,988.071687,12.0,170.69283,0.087510,12,gps,0.9,1.4,1.7,3.6,NaN,NaN,NaN,81,NaN,1.905784,1.002
2,1.577221e+09,-27.717818,-51.098886,988.190402,12.0,255.83438,0.116968,12,gps,0.9,1.5,1.7,3.6,NaN,NaN,NaN,81,NaN,1.372245,1.003
3,1.577221e+09,-27.717820,-51.098893,989.539177,16.0,308.75323,0.049259,12,gps,0.9,1.4,1.6,3.6,NaN,NaN,NaN,81,NaN,0.720180,0.988
4,1.577221e+09,-27.717869,-51.098902,986.509513,16.0,NaN,0.000000,12,gps,0.8,1.3,1.5,3.6,NaN,NaN,NaN,81,NaN,5.482620,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1572,1.577222e+09,-27.717743,-51.099119,980.994608,4.0,131.76976,1.037004,22,gps,0.7,1.0,1.2,3.6,NaN,NaN,NaN,75,NaN,1.032037,1.000
1573,1.577222e+09,-27.717749,-51.099115,980.963261,6.0,129.76077,0.451316,22,gps,0.7,1.0,1.2,3.6,NaN,NaN,NaN,75,NaN,0.754157,1.000
1574,1.577222e+09,-27.717746,-51.099114,981.304869,6.0,129.74986,0.014915,22,gps,0.7,1.0,1.2,3.6,NaN,NaN,NaN,75,NaN,0.289941,1.000
1575,1.577222e+09,-27.717743,-51.099115,981.537893,4.0,129.74384,0.014796,22,gps,0.7,1.0,1.2,3.6,NaN,NaN,NaN,75,NaN,0.335361,1.000


In [ ]:
df.columns

Index(['timestamp', 'latitude', 'longitude', 'elevation', 'accuracy',
       'bearing', 'speed_meters_per_second', 'satellites', 'provider', 'hdop',
       'vdop', 'pdop', 'geoidheight', 'ageofdgpsdata', 'dgpsid', 'activity',
       'battery', 'annotation', 'distance_meters', 'elapsed_time_seconds'],
      dtype='object')

In [ ]:
df.dtypes

timestamp                  float64
latitude                   float64
longitude                  float64
elevation                  float64
accuracy                   float64
bearing                    float64
speed_meters_per_second    float64
satellites                   int64
provider                    object
hdop                       float64
vdop                       float64
pdop                       float64
geoidheight                float64
ageofdgpsdata              float64
dgpsid                     float64
activity                   float64
battery                      int64
annotation                 float64
distance_meters            float64
elapsed_time_seconds       float64
dtype: object

In [ ]:
columns

['timestamp',
 'latitude',
 'longitude',
 'elevation',
 'accuracy',
 'bearing',
 'speed_meters_per_second',
 'satellites',
 'provider',
 'hdop',
 'vdop',
 'pdop',
 'geoidheight',
 'ageofdgpsdata',
 'dgpsid',
 'activity',
 'battery',
 'annotation',
 'distance_meters',
 'elapsed_time_seconds']

In [ ]:
        for column in columns:
            schema["properties"].update(
                {
                    f"{column}": {
                        "description": f"generic {column} ",
                        "type": "string"
                    }
                }
            )


In [ ]:
for column in columns:


In [ ]:
    def get_schema_to_produce_consume_data(cls, file_path):
        columns = next(pd.read_csv(file_path, chunksize=10)).columns

        schema = dict()
        schema.update({
            "$id": "http://example.com/myURI.schema.json",
            "$schema": "http://json-schema.org/draft-07/schema#",
            "additionalProperties": False,
            "description": "Sample schema to help you get started.",
            "properties": dict(),
            "title": "SampleRecord",
            "type": "object"})
        for column in columns:
            schema["properties"].update(
                {
                    f"{column}": {
                        "description": f"generic {column} ",
                        "type": "string"
                    }
                }
            )


        schema = json.dumps(schema)

        print(schema)
        return schema

In [ ]:
schema = {}

In [ ]:
schema.update({ "$id": "http://example.com/myURI.schema.json",
               "$schema": "http://json-schema.org/draft-07/schema#",
                 "additionalProperties": False,
                "description": "Sample schema to help you get started.",
                "properties" : {},
                "title": "SampleRecord",
                "type": "object"
               })

In [ ]:
for column in columns:
  schema["properties"].update(
      {
          f"{column}":{
              "description":f"generic{column}",
              "type":"string"
          }
      }
  )

In [ ]:
schema = json.dumps(schema)
print(schema)

{"$id": "http://example.com/myURI.schema.json", "$schema": "http://json-schema.org/draft-07/schema#", "additionalProperties": false, "description": "Sample schema to help you get started.", "properties": {"timestamp": {"description": "generictimestamp", "type": "string"}, "latitude": {"description": "genericlatitude", "type": "string"}, "longitude": {"description": "genericlongitude", "type": "string"}, "elevation": {"description": "genericelevation", "type": "string"}, "accuracy": {"description": "genericaccuracy", "type": "string"}, "bearing": {"description": "genericbearing", "type": "string"}, "speed_meters_per_second": {"description": "genericspeed_meters_per_second", "type": "string"}, "satellites": {"description": "genericsatellites", "type": "string"}, "provider": {"description": "genericprovider", "type": "string"}, "hdop": {"description": "generichdop", "type": "string"}, "vdop": {"description": "genericvdop", "type": "string"}, "pdop": {"description": "genericpdop", "type": 

In [ ]:
from confluent_kafka import Producer, SerializationContext, MessageField
from confluent_kafka.schema_registry import SchemaRegistryClient, SchemaRegistryConfig, JSONSerializer
from confluent_kafka.serialization import StringSerializer
from uuid import uuid4
import json

ImportError: cannot import name 'SerializationContext' from 'confluent_kafka' (/usr/local/lib/python3.10/dist-packages/confluent_kafka/__init__.py)